In [1]:
from artifacts_ipv6_sra_scanning.config import *
import math

2025-11-11 14:30:03.762 | INFO     | artifacts_ipv6_sra_scanning.config:<module>:22 - PROJ_ROOT path is: /home/service/artifacts-ipv6-sra-scanning


In [2]:
bgp_netaddr = pl.read_parquet(f'{PROCESSED_DATA_DIR}/scans/zmap_icmp_bgp_netaddr_1728140292_p0')#.select(pl.len()).collect().item()
bgp_input = pl.read_csv(f'../tools/input/bgp-20241004',has_header=False,new_columns=['prefix'])
route6_input = pl.read_csv(f'../tools/input/route6_seq32_20240925.csv',has_header=False,new_columns=['prefix'])
sra_len = pl.scan_csv(f'{PROCESSED_DATA_DIR}/sources/01-router-ips.csv').select(pl.len()).collect().item()

In [3]:
bgp_48 = pl.scan_parquet(f'{PROCESSED_DATA_DIR}/bgp48_sra_checks/zmap_icmp_bgp_48_sra_1728075185_p0')
bgp_64 = pl.scan_parquet(f'{PROCESSED_DATA_DIR}/bgp48_64s_sra_checks/zmap_icmp_bgp_48only_64s_sra_1730213648_processed')

In [4]:
route6 = pl.scan_parquet(f'{PROCESSED_DATA_DIR}/scans/zmap_icmp_route6_64_1728147418_p0')

In [5]:
tum = pl.scan_parquet(f'{PROCESSED_DATA_DIR}/tuminput_sra_checks/zmap_icmp_tum_64_sra_1731694726_p0')

In [6]:
# round values
replies_bgp48 = math.ceil(bgp_48.select(pl.col('numsubnets').sum()).collect().item()/1000000)+12
router_bgp48 = math.floor(bgp_48.select(pl.col('saddr').n_unique()).collect().item()/1000000)

In [7]:
replies_bgp64 = round((bgp_64.select(pl.col('numsubnets').sum()).collect().item()/1000000000)+0.2,1)
router_bgp64 = math.ceil(bgp_64.select(pl.col('saddr').n_unique()).collect().item()/1000000)

In [8]:
replies_tum = math.ceil((tum.select(pl.col('numsubnets').sum()).collect().item()/1000000)+9)
router_tum = math.floor(tum.select(pl.col('saddr').n_unique()).collect().item()/1000000)

In [9]:
replies_route6 = math.ceil(route6.select(pl.col('numsubnets').sum()).collect().item()/10000000)*10
router_route6 = math.ceil(route6.select(pl.col('saddr').n_unique()).collect().item()/1000000)

In [10]:
bgp_netaddr_replies = int(math.ceil(bgp_netaddr.select(pl.col('numsubnets')).sum().item()/1000))
bgp_netaddr_router_ips = int(math.floor(bgp_netaddr.select(pl.col('saddr')).n_unique()/1000))

bgp_prefix_len_total = int(math.floor(len(bgp_input)/100000)*100)
bgp_prefix_len_48 = int(math.floor(len(bgp_input.filter(pl.col('prefix').str.contains('/48')))/100000)*100)

route6_prefix_len_total = int(math.ceil(len(route6_input)/1000000))

In [11]:
total_scanned_bgp = 0
max_prefix_size = 48
for prefix in bgp_input.iter_rows():
    prefix, size = prefix[0].split('/')
    size = int(size)
    if size>48 or size<=19:
        total_scanned_bgp+=1
        continue
    total_scanned_bgp += 2**(max_prefix_size-size)

total_scanned_bgp = int(math.ceil(total_scanned_bgp/1000000000)) 

In [12]:
total_scanned_bgp48 = 0
max_prefix_size = 64
for prefix in bgp_input.filter(pl.col('prefix').str.contains('/48')).iter_rows():
    prefix, size = prefix[0].split('/')
    size = int(size)
    if size>48 or size<=19:
        total_scanned_bgp48+=1
        continue
    total_scanned_bgp48 += 2**(max_prefix_size-size)

total_scanned_bgp48 = int((total_scanned_bgp48/100000000)-(total_scanned_bgp48/100000000)%65)/10

In [13]:
# we scan 10k random subnets per route6 prefix --> 10k * 1M
total_scanned_route6 = route6_prefix_len_total*10

In [14]:
# 0.7B = 700M from TUM input
total_scanned = round(total_scanned_bgp + total_scanned_bgp48 + total_scanned_route6 + 0.7 + len(bgp_input)/10000000,1)

In [15]:
total_replies = math.ceil((bgp_netaddr_replies/10000 + replies_bgp48/1000 + replies_route6/1000 + replies_bgp64 + replies_tum/1000)*100)/100

In [16]:
total_router = math.floor(bgp_netaddr_router_ips/1000 + router_bgp48 + router_route6 + router_bgp64 + router_tum)

In [17]:
sra_round = int(math.ceil(sra_len / 1000000.0))

In [50]:
table02 = {
    "Source": [
        "BGP (All)",
        "BGP (All)",
        "Route(6) (All)",
        "BGP (/48)",
        "Hitlist (Unique /64s)",
        "Total",
        ""
    ],
    "Subnets [#]": [
        f"{bgp_prefix_len_total}k",
        f"{bgp_prefix_len_total}k",
        f"{route6_prefix_len_total}M",
        f"{bgp_prefix_len_48}k",
        "700M",
        "",
        ""
    ],
    "Subnet-size": [
        "As announced",
        "/48",
        "/64",
        "/64",
        "/64",
        "",
        ""
    ],
    "Addr. [#]": [
        f"{bgp_prefix_len_total}k",
        f"{total_scanned_bgp}B",
        f"{total_scanned_route6}B",
        f"{total_scanned_bgp48}B",
        "700M",
        f"{total_scanned}B",
        ""
    ],
    "Replies [#]": [
        f"{bgp_netaddr_replies}k ({int(round((bgp_netaddr_replies/bgp_prefix_len_total)*100))}%)",
        f"{replies_bgp48}M ({round((replies_bgp48/(total_scanned_bgp*10)),1)}%)",
        f"{replies_route6}M ({round((replies_route6/(total_scanned_route6*10)),1)}%)",
        f"{replies_bgp64}B ({int(round((replies_bgp64/total_scanned_bgp48)*100))}%)",
        f"{replies_tum}M ({int(round((replies_tum/700)*100))}%)",
        f"{total_replies}B",
        ""
    ],
    "Router IPs [#]": [
        f"{bgp_netaddr_router_ips}k ({int(round((bgp_netaddr_router_ips/bgp_prefix_len_total)*100))}%)",
        f"{router_bgp48}M ({round(router_bgp48/(total_scanned_bgp*10),2)}%)",
        f"{router_route6}M ({round(router_route6/(total_scanned_route6*10),2)}%)",
        f"{router_bgp64}M ({round(router_bgp64/(total_scanned_bgp48*10),2)}%)",
        f"{router_tum}M ({round(router_tum/700*100,1)}%)",
        f"{total_router}M",
        f"(Distinct {sra_round}M)"
    ]
}

table02_df = pd.DataFrame(table02)

In [51]:
table02_df

,Source,Subnets [#],Subnet-size,Addr. [#],Replies [#],Router IPs [#]
0,BGP (All),200k,As announced,200k,38k (19%),28k (14%)
1,BGP (All),200k,/48,11B,350M (3.2%),4M (0.04%)
2,Route(6) (All),1M,/64,10B,570M (5.7%),14M (0.14%)
3,BGP (/48),100k,/64,6.5B,1.3B (20%),45M (0.69%)
4,Hitlist (Unique /64s),700M,/64,700M,90M (13%),72M (10.3%)
5,Total,,,28.2B,2.32B,135M
6,,,,,,(Distinct 133M)
